In [25]:
'''
TWOTOGGLE_CASE.PY - Consider the case of the Punisher circuit with two toggle switches.
'''
# By Kirill Sechkar

# PACKAGE IMPORTS 
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 

# OWN CODE IMPORTS
import synthetic_circuits as circuits
from cell_model import *

cpu


Loading BokehJS ...

In [26]:
# INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.punisher_xtra_initialise,
    circuits.punisher_xtra_ode,
    circuits.punisher_xtra_F_calc,
    par, init_conds)  # load the circuit

In [27]:
# INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.twotoggles_punisher_initialise,
    circuits.twotoggles_punisher_ode,
    circuits.twotoggles_punisher_F_calc,
    par, init_conds)  # load the circuit

In [28]:
# PARAMETERISE THE CIRCUIT

 # TOGGLE SWITCHES
for togswitchnum in (1, 2):  # cycle through toggle switches
    for toggenenum in (1, 2):  # cycle through the genes of the current switch
        par['c_tog' + str(togswitchnum) + str(toggenenum)] = 1  # copy no. (nM)
        par['a_tog' + str(togswitchnum) + str(toggenenum)] = 1e5  # promoter strength (unitless)

        # transcription regulation function
        reg_func_string = 'dna(tog' + str(togswitchnum) + str(toggenenum) + '):p_tog' + str(togswitchnum) + str(
            (toggenenum - 2) % 2 + 1)  # dna(rep1):p_rep3, dna(rep2):p_rep1 or dna(rep3):p_rep2
        par['K_' + reg_func_string] = 5000  # half-saturation constant
        par['eta_' + reg_func_string] = 2  # Hill coefficient
        par['baseline_tog' + str(togswitchnum) + str(
            toggenenum)] = 0.025  # baseline transcription activation due to leakiness
        par['p_tog' + str(togswitchnum) + str(
            toggenenum) + '_ac_frac'] = 1  # active fraction of protein (i.e. share of molecules not bound by the inducer)
    # break symmetry for each of the toggle switches
    init_conds['m_tog' + str(togswitchnum) + '1'] = 4000

# PUNISHER
# switch gene conc
par['c_switch'] = 10.0  # gene concentration (nM)
par['a_switch'] = 400.0  # promoter strength (unitless)
par['d_switch']=0.01836
# integrase - expressed from the switch gene's operon, not its own gene => c_int, a_int irrelevant
par['k+_int'] = par['k+_switch']/80.0  # RBS weaker than for the switch gene
par['d_int'] = 0.0#0.01836 # rate of integrase degradation per protease molecule (1/nM/h)
# CAT (antibiotic resistance) gene
init_conds['cat_pb'] = 10.0  # gene concentration (nM) - INITIAL CONDITION< NOT PARAMETER as it can be cut out by the integrase
par['a_cat'] = 500.0  # promoter strength (unitless)
# synthetic protease gene
par['c_prot'] = 10.0  # gene concentration (nM)
par['a_prot'] = 25.0  # promoter strength (unitless)
init_conds['p_prot'] = 1500.0 # if zero at start, the punisher's triggered prematurely

# punisher's transcription regulation function
par['K_switch'] = 300.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 2 # Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
par['baseline_switch_alt'] = 0
par['p_switch_ac_frac'] = 0.85  # active fraction of protein (i.e. share of molecules bound by the inducer)

# CULTURE MEDIUM
init_conds['s'] = 0.5
par['h_ext'] = 10.5 * (10.0 ** 3)

# par['k_sxf'] = 0.0  # no synthetic gene expression loss

In [ ]:
# ECC 2024 - ILLUSTRATE PARAMETER TUNING - COMMENT OUT IRRELEVANT BITS

# ADJUSTED THRESHOLD

# SLOWING DOWN THE TIMESCALE
par['a_prot'] = par['a_prot']*0.2  # promoter strength (unitless)
par['p_switch_ac_frac'] = 0.276  # active fraction of protein (i.e. share of molecules bound by the inducer)



In [29]:
# SET UP THE DETERMINISTIC SIMULATION PARAMETERS

# diffrax simulator
savetimestep = 0.1  # save time step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver

# simulation time frames
tf_prepun = (0,25) # time frame for simulation before punisher comes online - to avoid premature tiggering
tf_pun = (25,50) # time frame for simulation after punisher comes online
tf_pulse = (50,51) # time frame for inducer pulse that flips the toggle
tf_afterpulse = (51,100) # time frame for simulation after the pulse

# inducer pulse parameters
pulse_ptog11_acfrac = 0.0 # active fraction of p_tog11 during the inducer pulse
pulse_ptog21_acfrac = 0.0 # active fraction of p_tog21 during the inducer pulse

In [30]:
# SIMULATE THE FLIPPING OF TOGGLES

 # initial simulation to get the steady state without gene expression loss
ac_frac = par['p_switch_ac_frac']
par['p_switch_ac_frac'] = 0.0  # first with punisher inactive to avoid it being triggered
sol = ode_sim(par,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf_prepun, jnp.arange(tf_prepun[0], tf_prepun[1], savetimestep),  # time frame and time axis for saving the system's state
              rtol, atol)  # relative and
ts = np.array(sol.ts)
xs = np.array(sol.ys)
par['p_switch_ac_frac'] = ac_frac
sol_punisher_active = ode_sim(par,  # dictionary with model parameters
                              ode_with_circuit,  # ODE function for the cell with synthetic circuit
                              sol.ys[-1, :],
                              # initial condition VECTOR
                              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                              cellmodel_auxil.synth_gene_params_for_jax(par, circuit_genes),
                              # synthetic gene parameters for calculating k values
                              tf_pun, jnp.arange(tf_pun[0], tf_pun[1], savetimestep),
                              # time frame and time axis for saving the system's state
                              rtol, atol)  # relative and
ts = np.concatenate((ts, np.array(sol_punisher_active.ts)), axis=0)
xs = np.concatenate((xs, np.array(sol_punisher_active.ys)), axis=0)

# par['func_tog11'] = 0.0 
# par['func_tog12'] = 0.0
# par['func_tog21'] = 0.0
# par['func_tog22'] = 0.0
# simulate toggle switch flipping by inducer pulse
orig_ptog11_acfrac = par['p_tog11_ac_frac']  # save the original active fraction of p_tog11
orig_ptog21_acfrac = par['p_tog21_ac_frac']  # save the original active fraction of p_tog21
# par['p_tog11_ac_frac'] = pulse_ptog11_acfrac  # set the active fraction of p_tog11 to the pulse value
# par['p_tog21_ac_frac'] = pulse_ptog21_acfrac  # set the active fraction of p_tog21 to the pulse value
x0_pulse = sol_punisher_active.ys[-1, :]  # simulation will resume from the last time point
sol_pulse = ode_sim(par,  # dictionary with model parameters
                        ode_with_circuit,  # ODE function for the cell with synthetic circuit
                        x0_pulse,  # initial condition VECTOR
                        len(circuit_genes), len(circuit_miscs), circuit_name2pos,  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                        cellmodel_auxil.synth_gene_params_for_jax(par, circuit_genes), # synthetic gene parameters for calculating k values
                        tf_pulse, jnp.arange(tf_pulse[0], tf_pulse[1], savetimestep),  # time frame and time axis for saving the system's state
                        rtol, atol)  # relative and absolute tolerances
ts = np.concatenate((ts, np.array(sol_pulse.ts)), axis=0)
xs = np.concatenate((xs, np.array(sol_pulse.ys)), axis=0)

# after the pulse, inducer levels return to the original state
par['p_tog11_ac_frac'] = orig_ptog11_acfrac  # set the active fraction of p_tog11 to the pulse value
par['p_tog21_ac_frac'] = orig_ptog21_acfrac  # set the active fraction of p_tog21 to the pulse value
x0_afterpulse = sol_pulse.ys[-1, :]  # simulation will resume from the last time point
sol_afterpulse = ode_sim(par,  # dictionary with model parameters
                        ode_with_circuit,  # ODE function for the cell with synthetic circuit
                        x0_afterpulse,  # initial condition VECTOR
                        len(circuit_genes), len(circuit_miscs), circuit_name2pos,  # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
                        cellmodel_auxil.synth_gene_params_for_jax(par, circuit_genes),  # synthetic gene parameters for calculating k values
                        tf_afterpulse, jnp.arange(tf_afterpulse[0], tf_afterpulse[1], savetimestep),  # time frame and time axis for saving the system's state
                        rtol, atol)  # relative and absolute tolerances
ts = np.concatenate((ts, np.array(sol_afterpulse.ts)), axis=0)
xs = np.concatenate((xs, np.array(sol_afterpulse.ys)), axis=0)

In [31]:
# PLOT THE CELL MODEL TRAJECTORIES

# cell protein mass breakdown
mass_fig=cellmodel_auxil.plot_protein_masses(ts,xs,par,circuit_genes) 

# native mRNA, tRNA & protein levels, chloramphenicol level
nat_mrna_fig,nat_prot_fig,nat_trna_fig,h_fig = cellmodel_auxil.plot_native_concentrations(ts, xs, par, circuit_genes)  

# cell growth, translation elongation, ribosome synth. regulation, ppGpp level, tRNA aminoacylation, Resource Competition Denominator
l_figure, e_figure, Fr_figure, ppGpp_figure, nu_figure, D_figure = cellmodel_auxil.plot_phys_variables(ts, xs, par, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                                                       dimensions=(320,240))  

# show plots
bkplot.show(bklayouts.grid([[mass_fig, nat_mrna_fig, nat_prot_fig],
                            [nat_trna_fig, h_fig, l_figure],
                            [e_figure, Fr_figure, D_figure]]))

In [32]:
# PLOT THE CIRCUIT TRAJECTORIES

# synthetic mRNA and protein levels, different states of CAT gene DNA
het_mrna_fig, het_prot_fig, misc_fig = cellmodel_auxil.plot_circuit_concentrations(ts, xs, par, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles,
                                                                                   dimensions=(320,240))

# synthetic gene regulation
F_fig = cellmodel_auxil.plot_circuit_regulation(ts, xs, circuit_F_calc, par, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles)

# show plots
bkplot.show(bklayouts.grid([[het_mrna_fig, het_prot_fig, misc_fig],
                            [None, None, None]]))